In [ ]:
data_one_id = data_prep.xs(226000188, level='WELL_ID')
data_one_id = data_one_id.drop(['targ_press_d','targ_press_h','targ_press_w'],axis=1)

In [ ]:
corr_matrix = data_one_id.corr()
high_corr = [
    column for column in lower.columns if any((lower[column] > 0.6)|(lower[column] < -0.6))
]

In [ ]:
other_features = [col for col in data_one_id.columns if col not in high_corr and col!='quality']


In [ ]:
data_features = data_one_id.copy()


In [ ]:
comparison_table, std_outliers = get_column_outliers(data_features)
anomalies_report(std_outliers)

In [ ]:
comparison_table, iqr_outliers = get_column_outliers(data_features, function=outlier_iqr)


In [ ]:
scaler = RobustScaler()
scaled_data = pd.DataFrame(
    data=scaler.fit_transform(data_features), 
    columns=data_features.columns
)

In [ ]:
euclidian_model = DistanceOutliers(metric='euclidean', percentile=90)
euclidian_model.fit(scaled_data)
euclidian_outliers = euclidian_model.predict(scaled_data)

In [ ]:
#DBSCAN
# для начала считаем все наблюдения аномальными
outlier_percentage = 1.

num_clusters = []
anomaly_percentage = []

# берем маленький эпсилон и начинаем увеличивать
eps = 0.05
eps_history = [eps]
while outlier_percentage>0.05:    
    model = DBSCAN(eps=eps).fit(scaled_data)
    labels = model.labels_
    num_clusters.append(len(np.unique(labels))-1)
    labels = np.array([1 if label == -1 else 0 for label in labels])
    # считаем текущий процент "шума"
    outlier_percentage = sum(labels==1) / len(labels)    
    eps += 0.05
    eps_history.append(eps)
    anomaly_percentage.append(outlier_percentage)
    
model = DBSCAN(eps)
model.fit(scaled_data)
density_outlier = np.array([1 if label == -1 else 0 for label in model.labels_])

model = DBSCAN(1.3)
model.fit(scaled_data)
density_outlier = np.array([1 if label == -1 else 0 for label in model.labels_])

In [ ]:
#SVM
one_class_svm = OneClassSVM(nu=0.1, gamma='auto')
one_class_svm.fit(scaled_data)
svm_outliers = one_class_svm.predict(scaled_data)
svm_outliers = np.array([1 if label == -1 else 0 for label in svm_outliers])

In [ ]:
#Iforest
isolation_forest = IsolationForest(n_estimators=100, contamination=0.1, 
                                   max_features=1.0, bootstrap=True, behaviour="new")
isolation_forest.fit(scaled_data)

isolation_outliers = isolation_forest.predict(scaled_data)
isolation_outliers = np.array([1 if label == -1 else 0 for label in isolation_outliers])

In [ ]:
summary = np.concatenate((
    [std_outliers], 
    [iqr_outliers], 
    [euclidian_outliers], 
#     [cityblock_outliers],
    [density_outlier],
    [svm_outliers],
    [isolation_outliers]
))

In [ ]:
summary = pd.DataFrame(
    summary.T, 
    columns=['std', 'iqr', 'euclid', 
#              'cityblock',
             'dbscan', 'svm', 'isolation']
)
summary.head()

In [ ]:
data_one_id_y = data_one_id_y.reset_index()
comparsion_with_y_outline = summary.copy()
comparsion_with_y_outline['2sigma'] = data_one_id_y['2sigma']
comparsion_with_y_outline['3sigma'] = data_one_id_y['3sigma']

comparsion_with_y_outline

In [ ]:
comparsion_with_y_outline = labeled_data['outlier_score']
comparsion_with_y_outline = pd.DataFrame(comparsion_with_y_outline)
comparsion_with_y_outline['2sigma'] = data_one_id_y['2sigma']
comparsion_with_y_outline['3sigma'] = data_one_id_y['3sigma']

abod	Angle-base Outlier Detection	pyod.models.abod.ABOD
cluster	Clustering-Based Local Outlier	pyod.models.cblof.CBLOF
cof	Connectivity-Based Local Outlier	pyod.models.cof.COF
iforest	Isolation Forest	pyod.models.iforest.IForest
histogram	Histogram-based Outlier Detection	pyod.models.hbos.HBOS
knn	K-Nearest Neighbors Detector	pyod.models.knn.KNN
lof	Local Outlier Factor	pyod.models.lof.LOF
svm	One-class SVM detector	pyod.models.ocsvm.OCSVM
pca	Principal Component Analysis	pyod.models.pca.PCA
mcd	Minimum Covariance Determinant	pyod.models.mcd.MCD
sod	Subspace Outlier Detection	pyod.models.sod.SOD
sos	Stochastic Outlier Selection	pyod.models.sos.SOS

cluster	Clustering-Based Local Outlier	pyod.models.cblof.CBLOF


sos	Stochastic Outlier Selection	pyod.models.sos.SOS

In [ ]:
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
result_score = pd.DataFrame(index = iforest_results.index)
result_score['iforest_an'] = iforest_results['Anomaly']
result_score['iforest_an_sc'] = iforest_results['Anomaly_Score']
result_score 

In [ ]:
# train model abod
abod = create_model('abod', fraction = 0.1)
abod_results = assign_model(abod)

In [ ]:
result_score['abod_an'] = abod_results['Anomaly']
result_score['abod_an_sc'] = abod_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
cluster = create_model('cluster', fraction = 0.1)
cluster_results = assign_model(cluster)

In [ ]:
result_score['cluster_an'] = cluster_results['Anomaly']
result_score['cluster_an_sc'] = cluster_results['Anomaly_Score']
result_score

In [ ]:
# train model iforest
cof = create_model('cof', fraction = 0.1)
cof_results = assign_model(cof)
cof_results.head()

In [ ]:
result_score['cof_an'] = cof_results['Anomaly']
result_score['cof_an_sc'] = cof_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
histogram = create_model('histogram', fraction = 0.1)
histogram_results = assign_model(histogram)

In [ ]:
result_score['histogram_an'] = histogram_results['Anomaly']
result_score['histogram_an_sc'] = histogram_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
knn = create_model('knn', fraction = 0.1)
knn_results = assign_model(knn)

In [ ]:
result_score['knn_an'] = knn_results['Anomaly']
result_score['knn_an_sc'] = knn_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
lof = create_model('lof', fraction = 0.1)
lof_results = assign_model(lof)

In [ ]:
result_score['lof_an'] = lof_results['Anomaly']
result_score['lof_an_sc'] = lof_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
svm = create_model('svm', fraction = 0.1)
svm_results = assign_model(svm)

In [ ]:
result_score['svm_an'] = svm_results['Anomaly']
result_score['svm_an_sc'] = svm_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
pca = create_model('pca', fraction = 0.1)
pca_results = assign_model(pca)

In [ ]:
result_score['pca_an'] = pca_results['Anomaly']
result_score['pca_an_sc'] = pca_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
mcd = create_model('mcd', fraction = 0.1)
mcd_results = assign_model(mcd)

In [ ]:
result_score['mcd_an'] = mcd_results['Anomaly']
result_score['mcd_an_sc'] = mcd_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
sod = create_model('sod', fraction = 0.1)
sod_results = assign_model(sod)

In [ ]:
result_score['sod_an'] = sod_results['Anomaly']
result_score['sod_an_sc'] = sod_results['Anomaly_Score']
result_score

In [ ]:
# train model abod
sos = create_model('sos', fraction = 0.1)
sos_results = assign_model(sos)

In [ ]:
result_score['sos_an'] = sos_results['Anomaly']
result_score['sos_an_sc'] = sos_results['Anomaly_Score']
result_score